## 기본 Agent 생성 및 실행

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
agent_model = init_chat_model("gpt-4o-mini")

/Users/kimsiyeon/Desktop/gaida-2/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
system_prompt = "당신은 웹 검색과 계산을 도와주는 AI 어시스턴트입니다. "

In [4]:
from langchain.tools import tool

@tool
def search_web(query: str) -> str:
    """웹에서 정보를 검색합니다."""
    return f"'{query}'에 대한 검색 결과입니다."

In [5]:
from langchain.agents import create_agent

ex_agent = create_agent(
    model = agent_model,
    tools = [search_web],
    system_prompt=system_prompt
)

In [50]:
query = '2024년 한국 GDP 얼마야?'
result = ex_agent.invoke({ "messages": [{'role':'user', 'content':query}]})
print(result["messages"][-1].pretty_print())

ex_agent.invoke({ "messages": [{'role':'user', 'content':query}]})

================================== Ai Message ==================================

현재 2024년 한국의 GDP에 대한 구체적인 예측치를 찾지 못했습니다. 그러나 보통 한국의 GDP는 경제 성장률에 따라 변동하기 때문에, 최근의 경제 전망이나 보고서를 참고하시거나 한국은행 또는 국제통화기금(IMF)의 공식 발표를 확인하는 것이 좋습니다. 추가적인 정보가 필요하시면 다시 요청해 주시기 바랍니다.
None


{'messages': [HumanMessage(content='2024년 한국 GDP 얼마야?', additional_kwargs={}, response_metadata={}, id='55e360fa-1eae-4435-9a4c-9e12b147ca0f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 71, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DDNN820cJQyvhheuJbdgacDYFkh7V', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c983a-36f5-7fb0-a871-95da5245434b-0', tool_calls=[{'name': 'search_web', 'args': {'query': '2024 South Korea GDP forecast'}, 'id': 'call_oy5dOIzSTYAYVqx1TMnMhYFu', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens':

In [7]:
query = '2024년 한국 GDP 얼마야?'
result = ex_agent.stream(
    {"messages": [{'role':'user','content':query }]}, stream_mode= 'values'
)

for event in result:
    print(event["messages"][-1].pretty_print())

================================ Human Message =================================

2024년 한국 GDP 얼마야?
None
================================== Ai Message ==================================
Tool Calls:
  search_web (call_5oYnlf7DJGq7JqqEIpmGu3IV)
 Call ID: call_5oYnlf7DJGq7JqqEIpmGu3IV
  Args:
    query: 2024 South Korea GDP forecast
None
================================= Tool Message =================================
Name: search_web

'2024 South Korea GDP forecast'에 대한 검색 결과입니다.
None
================================== Ai Message ==================================
Tool Calls:
  search_web (call_MDa3hmssrFjn6sRROgLUIVkw)
 Call ID: call_MDa3hmssrFjn6sRROgLUIVkw
  Args:
    query: 2024 South Korea GDP estimate
None
================================= Tool Message =================================
Name: search_web

'2024 South Korea GDP estimate'에 대한 검색 결과입니다.
None
================================== Ai Message ==================================
Tool Calls:
  search_web (call_yP5ed7y6MtwNXuZTIPU

In [8]:
query = '2024년 한국 GDP 얼마야?'
result = ex_agent.stream(
    {"messages": [{'role':'user','content':query }]}, stream_mode= 'values'
)

for event in result:
    print(event["messages"][-1].pretty_print())

================================ Human Message =================================

2024년 한국 GDP 얼마야?
None
================================== Ai Message ==================================
Tool Calls:
  search_web (call_VpuDsZ2nqv7Ow9PoZ3O18951)
 Call ID: call_VpuDsZ2nqv7Ow9PoZ3O18951
  Args:
    query: 2024 South Korea GDP forecast
None
================================= Tool Message =================================
Name: search_web

'2024 South Korea GDP forecast'에 대한 검색 결과입니다.
None
================================== Ai Message ==================================
Tool Calls:
  search_web (call_phwmuptIZP70Gj0i7zMc7wB1)
 Call ID: call_phwmuptIZP70Gj0i7zMc7wB1
  Args:
    query: 2024 South Korea GDP prediction
None
================================= Tool Message =================================
Name: search_web

'2024 South Korea GDP prediction'에 대한 검색 결과입니다.
None
================================== Ai Message ==================================
Tool Calls:
  search_web (call_07ToRLYQqciYc8m

In [9]:
query = '2024년 한국 GDP 얼마야?'

for chunk in ex_agent.stream(
    {"messages": [{'role': 'user', 'content': query}]}, stream_mode='values'
):
    last = chunk["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        print("아직 진행 중 (도구 호출 예정)")
    else:
        print("최종 답변:", last.content)


'''
hasattr(last, "tool_calls") -> 객체에  tool_calls 속성이 있는지 확인 -> Human Message, Tool Message는 tool_calls 속성 없음
참고) Tool Message에는 tool_calls가 없고 tool_call_id만 있음 

last.tool_calls -> 해당 객체에 tool_calls 리스트가 비어 있는지 확인 -> Ai Message중 일반텍스트(최종 답변)만 리스트 비어 있음 
참고) Ai Message에는 툴 호출 메시지, 일반텍스트(최종 답변)가 있음

'''
 

최종 답변: 2024년 한국 GDP 얼마야?
아직 진행 중 (도구 호출 예정)
최종 답변: '2024 South Korea GDP'에 대한 검색 결과입니다.
아직 진행 중 (도구 호출 예정)
최종 답변: '2024년 한국 GDP 예상'에 대한 검색 결과입니다.
아직 진행 중 (도구 호출 예정)
최종 답변: '2024 South Korea GDP forecast'에 대한 검색 결과입니다.
최종 답변: 2024년 한국의 GDP에 대한 정보는 아직 정확하게 확정되지 않았지만, 여러 경제 기관에서 예상치를 발표하고 있습니다. 대체로, 한국의 GDP는 약 1.8%에서 3% 사이 성장할 것으로 보입니다. 구체적인 수치는 기관이나 보고서에 따라 다를 수 있으니, 자세한 정보를 원하신다면 최신 경제 보고서를 참조하는 것이 좋습니다.


'\nhasattr(last, "tool_calls") -> 객체에  tool_calls 속성이 있는지 확인 -> Human Message, Tool Message는 tool_calls 속성 없음\n참고) Tool Message에는 tool_calls가 없고 tool_call_id만 있음 \n\nlast.tool_calls -> 해당 객체에 tool_calls 리스트가 비어 있는지 확인 -> Ai Message중 일반텍스트(최종 답변)만 리스트 비어 있음 \n참고) Ai Message에는 툴 호출 메시지, 일반텍스트(최종 답변)가 있음\n\n'

In [10]:
query = '2024년 한국 GDP 얼마야?'
for chunk in ex_agent.stream(
    {"messages": [{'role': 'user', 'content': query}]}, stream_mode='values'
):
    last = chunk["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        print("아직 진행 중 (도구 호출 예정)")
    elif last.type == "ai" and last.content:
        print("최종 답변:", last.content)

'''
첫번째 조건이 tool_calls 속성이 있고 그 tool_calls 리스트가 비어있지 않은 경우 -> Ai Messages (툴 호출 결정)만 분기됨
두번째 조건은 ai 타입일때이니 -> Ai message 중 최종 결과 답변임 
'''

아직 진행 중 (도구 호출 예정)
아직 진행 중 (도구 호출 예정)
최종 답변: 2024년 한국의 GDP에 대한 정보가 아직 구체적으로 확정되지 않았지만, 여러 예측에 따르면 한국의 GDP는 약 2조 1천억 달러에 이를 것으로 보입니다. 이는 경제 성장률이 약 3% 정도로 예상되기 때문입니다. 그러나 이러한 데이터는 시간이 지남에 따라 변동할 수 있습니다. 최신 정보와 예측치는 경제 컨설팅 기관이나 정부 통계를 참고하는 것이 좋습니다.


'\n첫번째 조건이 tool_calls 속성이 있고 그 tool_calls 리스트가 비어있지 않은 경우 -> Ai Messages (툴 호출 결정)만 분기됨\n두번째 조건은 ai 타입일때이니 -> Ai message 중 최종 결과 답변임 \n'

## RAG Agent 생성 및 실행

- pprint는 파이썬 범용 -> from pprint import pprint
- pretty_print() 는 랭체인 객체에서만 사용 가능 -> 랭체인 메서드 중 하나

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from pprint import pprint

file_path = "./AI_활용_패러다임_변화.pdf"
loader = PyPDFLoader(file_path)

# docs = [Document, Document, Document, ...] -> 페이지 수만큼
docs =loader.load()

# 1페이지의 content 및 metadata가 들어가져 있음 
docs[0]

'''
Document(  metadata = { 문서정보 dict },
           page_content = "1페이지 텍스트")
'''
# 2페이지 메타데이터 확인
pprint(docs[1].metadata)

# 11페이지 내용 확인 
pprint(docs[10].page_content)



{'creationdate': '2025-01-17T14:48:58+09:00',
 'creator': 'Hwp 2020 11.0.0.8658',
 'moddate': '2025-01-17T14:48:58+09:00',
 'page': 1,
 'page_label': '2',
 'pdfversion': '1.4',
 'producer': 'Hancom PDF 1.3.0.547',
 'source': './AI_활용_패러다임_변화.pdf',
 'total_pages': 18}
('GDX REPORT\n'
 '한국지능정보사회진흥원 ❘ 12\n'
 'n[Microsoft] 사용자가 별도의 코딩 지식없이 사용자 맞춤형 AI 에이전트를 생성할 수 있는 플랫폼인 ‘코파일럿 '
 '스튜디오(Copilot Studio)’ 출시§코파일럿 스튜디오는 마이크로소프트의 다양한 플랫폼(Office 365, Teams 등)과 '
 '통합되어, 기업 업무 환경에서 사용자가 생성한 AI 에이전트를 사용할 수 있도록 지원\xad사용자는 맞춤형 AI 에이전트를 직접 구축하여 '
 '사용자의 개입을 최소화하고 다양한 비즈니스 프로세스 효율화 가능※ 또한, 에이전트 라이브러리(Agent library)를 활용해 다양한 '
 '템플릿을 제공받아 쉽게 에이전트 설정\xad마이크로소프트 365와 Azure AI Foundry 통합으로 누구나 쉽게 AI 에이전트를 '
 '설계하고 관리할 수 있는 환경 제공※ Copilot: OpenAI와 협력하여 개발한 AI 비서로, 회의 요약, 이메일 작성 등의 작업을 '
 '수행Copilot Vision: 사용자의 화면을 함께 보며 다양한 질문에 답변하고 도움을 제공그림 5 |‘코파일럿 스튜디오’ 활용 예시\n'
 '* 자료 : Microsoft(2024)\xad자율 에이전트, 음성 지원, 고급 지식 조정 등 다양하고 새로운 기능을 통해 복잡한 업무를 '
 '자동화하여 비즈니스 효율성 극대화§마이크로소프트는 이그나이트 2024(Microsoft Ignite 2024

In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 다큐먼트 객체들 split -> chunk 분할 -> chunk 또한 다큐먼트 객체와 같음 (즉, page_content 와 metadata로 구성되어 있음)
text = text_splitter.split_documents(docs)
pprint(text[4].page_content)
pprint(text[4].metadata)


('Ⅰ. AI 에이전트 정의 및 현황AI 에이전트란 무엇인가?nAI 에이전트는 정해진 목표 달성을 위해 상황 파악, 워크플로우 계획 수립, '
 '외부 데이터와 분석 툴 활용 등으로 자율적 작업을 수행하는 지능형 시스템§챗GPT 출시 이후 생성형 AI 서비스가 빠르게 확산되며, '
 '기존의 AI 챗봇을 넘어 사용자의 지시에 따라 다양한 작업을 스스로 수행하는 ‘AI 에이전트’ 시대 본격화\xadAI 에이전트는 단순 '
 '정보 제공을 넘어 복잡한 의사 결정 지원과 작업 자동화를 통해 인간과의 협업 수준을 한층 더 발전시켜 효율성 향상에 기여할 것으로 기대※ '
 '주요 빅테크 기업의 AI 에이전트 정의  · AWS: AI 에이전트란 환경과 상호 작용하고, 데이터를 수집하고, 데이터를 사용하여 사전 '
 '결정된 목표를 달성하기 위해 필요한 작업을 스스로 결정해서 수행할 수 있는 소프트웨어 프로그램  · NVIDIA: 에이전틱 AI는 여러 '
 '출처로부터 방대한 양의 데이터를 수집해 독립적으로 문제 분석 및 전략을 개발하며, 공급망 최적화, 사이버 보안 취약성 분석, 시간 소모적 '
 '업무 지원 등을 수행하는 시스템§AI 에이전트는 정교한 추론 및 반복적인 계획을 통해 복잡한 문제를 자율적으로 해결하여 산업 전반의 '
 '생산성·효율성을 향상시킬 차세대 AI 기술로 평가\xadAI 에이전트는 인식, 처리, 행동, 학습 및 적용, 자율성의 5가지 주요 특징을 '
 '바탕으로 수행 및 문제 해결특징 내용인식(Perception)Ÿ 센서, 데이터 파일, 인터넷 등 다양한 데이터 수집방법을 통해 주변 환경 '
 '인식 및 파악처리(Brain)Ÿ 입력 데이터 기반 주요 정보 추출 및 새로운 정보학습을 통한 의사 결정행동(Action)Ÿ 도출된 결정, '
 '해답 기반 텍스트 생성, API 호출,물리적 동작 등 실제 수행학습 및 적용(Learning and Adaptation)Ÿ 경험 기반 '
 '지속적 학습, 과거 데이터를 기반으로미래 행동 개선자율성(Autonom

In [40]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# .embed_query -> [자연어 -> 숫자로 바꿔주는 메서드] (여기는 split한 다큐먼트의 page_content만 들어올 수 있음 -> 즉, 문자열만 들어 올 수 있음 )
check_vector = embeddings.embed_query(text[0].page_content)
pprint(check_vector)
len(check_vector)

[0.003114554565399885,
 -0.027200886979699135,
 -0.005877144634723663,
 0.05615495517849922,
 0.051400113850831985,
 -0.036577753722667694,
 -0.014649695716798306,
 0.02409297227859497,
 -0.00949640292674303,
 0.0017199779395014048,
 0.03803873807191849,
 -0.02004205994307995,
 -0.015898173674941063,
 -0.012776979245245457,
 0.0032440510112792253,
 -0.059873826801776886,
 -0.05310016870498657,
 0.005920310039073229,
 -0.007829551585018635,
 -0.011807415634393692,
 -0.03575428947806358,
 0.018448257818818092,
 0.0460343137383461,
 -0.03339014947414398,
 0.028449364006519318,
 0.0018594355788081884,
 0.026643056422472,
 -0.005817377008497715,
 0.03176978603005409,
 0.0018942999886348844,
 0.08128389716148376,
 -0.05477365851402283,
 0.010937465354800224,
 -0.06805533915758133,
 0.004479247611016035,
 0.04388267919421196,
 -0.011216381564736366,
 0.02333591692149639,
 0.0036956281401216984,
 -0.01362700667232275,
 0.03724183887243271,
 -0.03176978603005409,
 0.0042368569411337376,
 0.0386

1536

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# 청크들 벡터로 변환 및 저장 
result = vector_store.add_documents(text)


In [43]:

# 잘되나 확인 해보기  -> 벡터 값 유사도가 높은 청크를 봔환 해주는 메서드 (.similarity_search)
similar_docs = vector_store.similarity_search("ai 주요 구성", k=3)
print(similar_docs)



# 검색기 노드로 생성 해보기 
retriever = vector_store.as_retriever(
    search_type = 'similarity',   # 검색 방식 : 유사도 -> 없어도 default가 similarity임
    search_kwargs = {'k' : 3}     # 문서 검색 3개를 찾아라 (결과 개수 : 3개)
)

# invoke시 쿼리를 벡터화해서 유사한 문서를 찾아온다 
''' invoke() 내부 동작:
    문자열 입력 → 임베딩 모델로 벡터화 → 벡터스토어에서 유사 문서 검색 → Document 리스트 반환'''

retriever.invoke('나이키의 미국 영업점 개수?')  

from langchain.tools import tool

# 검색기를 툴로 생성해보기 
@tool
def search_vectorstore(query: str) -> str :
    """Retrieve info to help answer a query about ai agent"""

    # 검색기 대신 벡터스토어 바로 활용하기 (chunk 2개만 검색)
    search_chunks = vector_store.similarity_search(query, k=2)

    result = ''

    # chunk 데이터 2개만 가져와서 for문을 통해 page_content 내용만 하나로 합치는 과정 
    for chunk in search_chunks:
        result += chunk.page_content + '/n/n'
    
    return result

# 결과는 질문 벡터와 가장 유사도가 높은 청크 2개의 page_content를 합친 문자열 
# search_vectorstore("ai agent 활용도 알려줘") 

[Document(id='0038e449-c16a-4aed-bbe1-e9f129bb0450', metadata={'producer': 'Hancom PDF 1.3.0.547', 'creator': 'Hwp 2020 11.0.0.8658', 'creationdate': '2025-01-17T14:48:58+09:00', 'moddate': '2025-01-17T14:48:58+09:00', 'pdfversion': '1.4', 'source': './AI_활용_패러다임_변화.pdf', 'total_pages': 18, 'page': 3, 'page_label': '4'}, page_content='< 표 1 | AI 에이전트의 5가지 주요 특징 >\n     * 자료 : SK C&C(2024)'), Document(id='24d34592-f965-4a32-894e-b33231980a4f', metadata={'producer': 'Hancom PDF 1.3.0.547', 'creator': 'Hwp 2020 11.0.0.8658', 'creationdate': '2025-01-17T14:48:58+09:00', 'moddate': '2025-01-17T14:48:58+09:00', 'pdfversion': '1.4', 'source': './AI_활용_패러다임_변화.pdf', 'total_pages': 18, 'page': 6, 'page_label': '7'}, page_content='GDX REPORT\n한국지능정보사회진흥원 ❘ 8\nAI 에이전트 기술 발전과 주요 트렌드nAI 에이전트 기술은 특정 AI가 다른 AI 또는 사람과 팀을 이룬 형태로 다양한 과정을 통해 주어진 환경에 대응할 수 있도록 설계된 자율 시스템으로 발전§학습 데이터 기반으로 프롬프트에 간단히 응답한 LLMs에서부터 자율성·인간-AI간 상호작용이 강화된 AI Agent까지 기술이 빠르게 발전\xad(LLMs) 초기 대규모 언어모델은 방대한 학습 데이터 세트를 기반으로 주로 정보 생성과 

In [48]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
ai_agent = create_agent(

    model = ("openai:gpt-4o-mini"),
    tools = [search_vectorstore],
    system_prompt='''너는 search_vectorstore툴을 활용하여 사용자의 질문에 답변해주면서 몇페이지 참고했는지도 같이 답변해줘 단, 툴에 없는 내용이면 없다고 말해줘야 해'''
)


In [49]:
answer = ai_agent.invoke({
    "messages": [{'role':'user', 'content':"ai agent 중요성 알려줘"}]
})

answer["messages"][-1].content

'AI 에이전트는 특정 AI가 다른 AI 또는 사람과 팀을 이루어 주어진 환경에 효과적으로 대응할 수 있도록 설계된 자율 시스템입니다. AI 에이전트의 중요성은 다음과 같은 몇 가지 특성에서 나타납니다:\n\n1. **자율성**: AI 에이전트는 복잡한 문제를 스스로 이해하고 자율적으로 계획 및 실행할 수 있는 능력을 갖추고 있습니다.\n2. **강화된 상호작용**: 인간과 AI 간의 상호작용이 개선되어 보다 자연스러운 커뮤니케이션이 가능해집니다.\n3. **복잡한 문제 해결**: AI 에이전트는 하드코딩된 도구를 사용하여 복잡한 문제를 해결할 수 있는 체계를 가지고 있습니다.\n4. **의사 결정 능력**: 도구의 선택과 사용 시점을 자율적으로 결정하여 더 높은 수준의 의사 결정을 수행할 수 있습니다.\n\n이러한 특성 덕분에 AI 에이전트는 다양한 산업에서 혁신적인 변화를 가져오는 중요한 역할을 하게 됩니다.\n\n참고한 페이지: 8.'

## sql Agent 생성 및 실행
- `postgresql` 연결

In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
from langchain_community.utilities import SQLDatabase
import os
DB_URI = os.getenv("DB_URI")
db = SQLDatabase.from_uri(DB_URI)

db.dialect
db.get_usable_table_names()
db.run("SELECT * FROM employees LIMIT 5;")

"[(1, '김백엔드1', 'backend1@company.com', 1), (2, '김백엔드2', 'backend2@company.com', 1), (3, '김백엔드3', 'backend3@company.com', 1), (4, '김백엔드4', 'backend4@company.com', 1), (5, '김백엔드5', 'backend5@company.com', 1)]"

In [57]:
from langchain.chat_models import init_chat_model
ai_model = init_chat_model("gpt-4o-mini")

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=ai_model)
tools = toolkit.get_tools()


# 툴 목록 보기 
for tool in tools:
    print(f"{tool.name}:{tool.description}")


[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x156db4cd0>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x156db4cd0>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x156db4cd0>), QuerySQLCheckerTool(description='Use this tool to double check if your que

In [61]:
dialect = db.dialect
top_k = 5

system_prompt = f"""
You are an agent designed to interact with a SQL database.

IMPORTANT: You must only use the following tables:
- employees: Employee information
- teams: Team information
Do NOT query any tables other than the ones listed above.

Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""

In [62]:
from langchain.agents import create_agent

sql_agent = create_agent(
    model = ai_model,
    tools = tools,
    system_prompt=system_prompt
)


In [66]:
query = '각팀마다 직원들의 수를 알려줘'
result = sql_agent.stream({
    "messages": [{'role':'user','content':query}]
}, stream_mode='values')

for event in result:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

각팀마다 직원들의 수를 알려줘
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_DJUN9PWM9GuqBkwuUN7WLneS)
 Call ID: call_DJUN9PWM9GuqBkwuUN7WLneS
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, employees, lottery_infos, lotto_draws, members, sales, sample, students, students_courses, teams
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_ekQFKbluDRt74ZzOjHYKbUdr)
 Call ID: call_ekQFKbluDRt74ZzOjHYKbUdr
  Args:
    table_names: employees, teams
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employees (
	id SERIAL NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(150) NOT NULL, 
	team_id INTEGER NOT NULL, 
	CONSTRAINT 

In [68]:
# human in the loop -> 인터럽트 걸어보기 위해 agent 하나 더 생성

from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 

sql_agent_test = create_agent(
    model=ai_model,
    tools=tools,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "sql_db_query": {"allowed_decisions": ["approve", "reject"]}
            },
            description_prefix="Tool 실행 전에 승인을 기다림",
        ),
    ],
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)


# 사용자한테 보이는 문구 ->  description_prefix + 도구 이름 + 인자(sql문)가 합쳐져서 보여줌

In [ ]:
# 인터럽트 걸어보기 -> 결과는 sql_db_query 실행전에 메시지 멈춤
import uuid

config = {"configurable": {"thread_id":str(uuid.uuid4()) }} 

result = sql_agent_test.stream(
        { "messages": [{ "role": "user", "content": "HR팀 직원들의 이름 알려줘",}]},
        config=config ,
        stream_mode='values'
)

for event in result:
    event["messages"][-1].pretty_print()


================================ Human Message =================================

HR팀 직원들의 이름 알려줘
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_f12wA5UazVEYgVWUgIuI8w0z)
 Call ID: call_f12wA5UazVEYgVWUgIuI8w0z
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, employees, lottery_infos, lotto_draws, members, sales, sample, students, students_courses, teams
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_f8x5jFY9VEgcshCbqBEv7KPy)
 Call ID: call_f8x5jFY9VEgcshCbqBEv7KPy
  Args:
    table_names: employees, teams
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employees (
	id SERIAL NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(150) NOT NULL, 
	team_id INTEGER NOT NULL, 
	CONSTRAINT e

In [73]:
# 상태 확인 해보기
state = sql_agent_test.get_state(config)
pprint(state.next)          # 다음 실행할 노드
pprint(state.tasks)         # 인터럽트 정보 (여기에 description_prefix가 포함됨)

('HumanInTheLoopMiddleware.after_model',)
(PregelTask(id='62448978-7cf1-b14c-669b-1c32f555577b', name='HumanInTheLoopMiddleware.after_model', path=('__pregel_pull', 'HumanInTheLoopMiddleware.after_model'), error=None, interrupts=(Interrupt(value={'action_requests': [{'name': 'sql_db_query', 'args': {'query': "SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;"}, 'description': 'Tool 실행 전에 승인을 기다림\n\nTool: sql_db_query\nArgs: {\'query\': "SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = \'HR\' LIMIT 5;"}'}], 'review_configs': [{'action_name': 'sql_db_query', 'allowed_decisions': ['approve', 'reject']}]}, id='9502c06139799202e21d5aad595c701b'),), state=None, result=None),)


In [ ]:
import uuid

config = {"configurable": {"thread_id":str(uuid.uuid4()) }} 

result = sql_agent_test.invoke(
        { "messages": [{ "role": "user", "content": "HR팀 직원들의 이름 알려줘",}]},
        config=config 
)

In [ ]:

# { messages[HumanMessage, AI Message, Tool Message], __interrupt__()} <- 키가 서로 다름 

pprint(result['__interrupt__'])

# > [
# >    Interrupt(
# >       value={
# >          'action_requests': [
# >             {
# >                'name': 'execute_sql',
# >                'arguments': {'query': 'DELETE FROM records WHERE created_at < NOW() - INTERVAL \'30 days\';'},
# >                'description': 'Tool execution pending approval\n\nTool: execute_sql\nArgs: {...}'
# >             }
# >          ],
# >          'review_configs': [
# >             {
# >                'action_name': 'execute_sql',
# >                'allowed_decisions': ['approve', 'reject']
# >             }
# >          ]
# >       }
# >    )
# > ]


[Interrupt(value={'action_requests': [{'args': {'query': 'SELECT e.name\n'
                                                         'FROM employees e\n'
                                                         'JOIN teams t ON '
                                                         'e.team_id = t.id\n'
                                                         'WHERE t.name = '
                                                         "'HR' \n"
                                                         'ORDER BY e.name\n'
                                                         'LIMIT 5;'},
                                       'description': 'Tool 실행 전에 승인을 기다림\n'
                                                      '\n'
                                                      'Tool: sql_db_query\n'
                                                      "Args: {'query': "
                                                      '"SELECT e.name\\nFROM '
                                       

In [89]:
# 인터럽트 걸고 인터럽트 메시지 까지 출력해보기 

from langgraph.types import Command
import uuid

# config = {"configurable": {"thread_id":str(uuid.uuid4()) }} 

# 1) thread_id를 변수에 저장
my_thread_id = str(uuid.uuid4())

config = {"configurable": {"thread_id":str(my_thread_id) }} 


# 2) 실행 → 인터럽트 걸림
result = sql_agent_test.stream(
        { "messages": [{ "role": "user", "content": "HR팀 직원들의 이름 알려줘",}]},
        config=config ,
        stream_mode='values'
)

for event in result:
        if "__interrupt__" in event:
            print("INTERRUPTED:") 
            interrupt = event["__interrupt__"][0] 
            for request in interrupt.value["action_requests"]:
                print(request["description"])


result = sql_agent_test.stream(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config ,
        stream_mode='values'
)

for event in result:
        event["messages"][-1].pretty_print()

INTERRUPTED:
Tool 실행 전에 승인을 기다림

Tool: sql_db_query
Args: {'query': "SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;"}
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_OxvxTZQBja7T7lLqce4jmNsL)
 Call ID: call_OxvxTZQBja7T7lLqce4jmNsL
  Args:
    query: SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_OxvxTZQBja7T7lLqce4jmNsL)
 Call ID: call_OxvxTZQBja7T7lLqce4jmNsL
  Args:
    query: SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;
================================= Tool Message =================================
Name: sql_db_query

[('정인사1',), ('정인사2',), ('정인사3',), ('정인사4',), ('정인사5',)]
================================== Ai Message ==================================

HR팀 직원들의 이름은 

INTERRUPTED:
Tool 실행 전에 승인을 기다림

Tool: sql_db_query
Args: {'query': "SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;"}

================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_OxvxTZQBja7T7lLqce4jmNsL)
 Call ID: call_OxvxTZQBja7T7lLqce4jmNsL
  Args:
    query: SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_OxvxTZQBja7T7lLqce4jmNsL)
 Call ID: call_OxvxTZQBja7T7lLqce4jmNsL
  Args:
    query: SELECT e.name FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name = 'HR' LIMIT 5;
================================= Tool Message =================================
Name: sql_db_query

[('정인사1',), ('정인사2',), ('정인사3',), ('정인사4',), ('정인사5',)]
================================== Ai Message ==================================

HR팀 직원들의 이름은 다음과 같습니다:

1. 정인사1
2. 정인사2
3. 정인사3
4. 정인사4
5. 정인사5

In [ ]:
# 3) 인터럽트 확인 
print(my_thread_id) # thread_id 확인 가능 -> 8fd759a2-9a40-4c55-9d53-097b8e241548

8fd759a2-9a40-4c55-9d53-097b8e241548
